In [8]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 


def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)



def matrix_fractional_differentiation_linear_plus(x,w,b,a):
    
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    '''
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    '''
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test


#ifogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        t0, t1 = t.tensor([0.]),t.tensor([1.])
        t0 = t0.to(device)
        t1 = t1.to(device)
        y1 = t.where(input1 - target <= 0, t0, t1)
        y_pred_f = t.abs(input1 - target)
        grad_output1 = gamma(3)*y_pred_f**(2-a)/gamma(3-a)*y1
        y2 = t.where(input1 - target > 0, t0, t1)
        grad_output2 = 2*(input1 - target)*y2

        grad_output = (grad_output1 + grad_output2)/input1.numel()
        
        return grad_output,None
    
#ifogd_Linear
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a)   #IFOGD
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    

In [13]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
time1 = time.time()
slide_windows_size = 36  #i.e.,input length
stock_array = ['GSPC','DJI','IXIC','FTSE','GDAXI','N225']   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
#stock_array = ['ETTh1','ETTh2','ETTm1','ETTm2']   #ETTh1，ETTh2，ETTm1，ETTm2
for stock in stock_array:
    #stock = 'DJI'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
    pre_days = 48     #i.e.,prediction lengths,48,96,192,384
    while pre_days <= 384:  
        df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
        del df_DJIA['Date']
        #del df_DJIA['date']        #ETT
        sca_DJIA = scaler.fit_transform(df_DJIA)
        label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)
        #label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
        sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
        #label
        deq_DJIA = deque(maxlen=pre_days)
        DJIA_label = []
        for i in label:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==pre_days:
                DJIA_label.append(list(deq_DJIA))
        DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
        #X
        deq_DJIA = deque(maxlen=slide_windows_size)
        DJIA_X = []
        for i in sca_DJIA:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==slide_windows_size:
                DJIA_X.append(list(deq_DJIA))
        DJIA_X = t.Tensor(DJIA_X)
        X_DJIA, y_DJIA = DJIA_X.to(device), DJIA_label.to(device)
        X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1

        class Net(nn.Module):
            def __init__(self,):
                super(Net,self).__init__()
                self.linear1 = Linear(slide_windows_size*5,128)
                #self.linear1 = Linear(slide_windows_size*7,128)  #ETT
                self.linear2 = Linear(128,64)
                self.linear3 = Linear(64,32)
                self.linear4 = Linear(32,pre_days)
                self.w1,self.b1 = self.linear1.weight,self.linear1.bias
                self.w2,self.b2 = self.linear2.weight,self.linear2.bias
                self.w3,self.b3 = self.linear3.weight,self.linear3.bias
                self.w4,self.b4 = self.linear4.weight,self.linear4.bias

            def forward(self,x):
                x = x.view(-1,slide_windows_size*5)
                #x = x.view(-1,slide_windows_size*7)   #ETT
                y2 = Linear_fractional.apply(x,self.w1,self.b1)   
                y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
                y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
                y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
                return y5

        net = Net()
        torch.save(net.state_dict(), r'./model/Adam_Stock/'+stock+'_'+str(pre_days)+'_net.pth')    

        a = 0.6
        lr =5e-5   #adam:5e-5
        epoch = 1000
        batch_size = 256
        train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11 = [],[],[],[],[]  #collecting train set loss
        while a < 1.1:
            a = round(a + 0.1,1)
            best_loss = 10
            net = Net()
            net.load_state_dict(torch.load(r'./model/Adam_Stock/'+stock+'_'+str(pre_days)+'_net.pth'))
            optimizer = torch.optim.Adam(net.parameters(),lr=lr)
            net.to(device)
            for i in range(epoch):
                loss_sum = 0
                net.train()  
                start = np.random.randint(11,size=1)[0]  #Generate a random integer between 0 and 10
                while start+batch_size<len(X_train):
                    X_batch = X_train[start:start+batch_size]
                    y_batch = y_train[start:start+batch_size]
                    start = start+batch_size
                    out = net(X_batch)
                    loss = MSELoss_fractional.apply(out,y_batch) 
                    loss.backward()
                    loss_sum += loss
                    optimizer.step()
                    net.zero_grad()
                if a==0.7:
                    train_loss_07.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.8:
                    train_loss_08.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.9:
                    train_loss_09.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.0:
                    train_loss_10.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.1:
                    train_loss_11.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                net.eval()
                with torch.no_grad():
                    out = net(X_val)
                    MSE_val = MSE(y_val.cpu().detach().numpy(),out.cpu().detach().numpy())   #MSE，MAPE
                    if best_loss > MSE_val:
                        best_loss = MSE_val
                        torch.save(net.state_dict(), r'./model/Adam_Stock/'+stock+'_net_fractional_'+str(a).replace(".", "_")+'_'+str(pre_days)+'.pth')
                        print('epoch:{0}'.format(i+1)+'\n'+'Validation Loss:{0}'.format(MSE_val))
            print('the order'+str(a)+'finished!')
        if pre_days==48:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==96:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==192:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==384:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        pre_days = pre_days*2
time2 = time.time()
print(time2 - time1)

epoch:34
Validation Loss:7.450718879699707
epoch:35
Validation Loss:5.971609115600586
epoch:42
Validation Loss:4.866287708282471
epoch:44
Validation Loss:4.795087814331055
epoch:47
Validation Loss:3.688652992248535
0.4588832
epoch:51
Validation Loss:2.471050262451172
epoch:53
Validation Loss:2.3511109352111816
epoch:55
Validation Loss:2.2784981727600098
epoch:56
Validation Loss:2.2612597942352295
epoch:58
Validation Loss:2.2260303497314453
epoch:60
Validation Loss:1.8405827283859253
epoch:64
Validation Loss:1.7593483924865723
epoch:68
Validation Loss:1.5708459615707397
epoch:77
Validation Loss:1.4977781772613525
epoch:79
Validation Loss:1.4629223346710205
epoch:96
Validation Loss:1.4110318422317505
0.22149366
epoch:114
Validation Loss:1.369293451309204
epoch:118
Validation Loss:1.3619827032089233
epoch:122
Validation Loss:1.2846318483352661
epoch:124
Validation Loss:1.24373459815979
epoch:128
Validation Loss:1.1643962860107422
0.17155163
epoch:166
Validation Loss:1.1023825407028198
epo

epoch:280
Validation Loss:0.5579829216003418
epoch:294
Validation Loss:0.5485914349555969
epoch:297
Validation Loss:0.5005924701690674
0.12230893
epoch:308
Validation Loss:0.48533421754837036
epoch:323
Validation Loss:0.42898494005203247
epoch:331
Validation Loss:0.4259321093559265
epoch:332
Validation Loss:0.4240875542163849
epoch:337
Validation Loss:0.4138481318950653
epoch:338
Validation Loss:0.40900546312332153
epoch:341
Validation Loss:0.40430447459220886
epoch:349
Validation Loss:0.38656362891197205
0.11283598
epoch:353
Validation Loss:0.3700154721736908
epoch:363
Validation Loss:0.36693888902664185
epoch:365
Validation Loss:0.3472415804862976
epoch:367
Validation Loss:0.3438299000263214
epoch:373
Validation Loss:0.3284090757369995
epoch:374
Validation Loss:0.325715571641922
epoch:375
Validation Loss:0.32113713026046753
epoch:376
Validation Loss:0.3080701231956482
epoch:383
Validation Loss:0.2922687232494354
epoch:384
Validation Loss:0.27704092860221863
epoch:393
Validation Loss:

epoch:421
Validation Loss:0.24683935940265656
epoch:431
Validation Loss:0.2444378286600113
epoch:435
Validation Loss:0.23185601830482483
epoch:439
Validation Loss:0.23096515238285065
epoch:440
Validation Loss:0.22961686551570892
0.10072404
epoch:450
Validation Loss:0.22753989696502686
epoch:453
Validation Loss:0.2273835390806198
epoch:454
Validation Loss:0.2251732349395752
epoch:455
Validation Loss:0.22388729453086853
epoch:458
Validation Loss:0.2198491096496582
epoch:459
Validation Loss:0.2170383334159851
epoch:468
Validation Loss:0.2164640575647354
epoch:472
Validation Loss:0.2160654067993164
epoch:473
Validation Loss:0.21461209654808044
epoch:474
Validation Loss:0.21146655082702637
epoch:475
Validation Loss:0.208468958735466
epoch:489
Validation Loss:0.19138771295547485
0.09466675
epoch:507
Validation Loss:0.19083254039287567
epoch:520
Validation Loss:0.1779147982597351
epoch:543
Validation Loss:0.17766685783863068
epoch:546
Validation Loss:0.17709827423095703
epoch:549
Validation L

epoch:378
Validation Loss:0.14792005717754364
epoch:383
Validation Loss:0.1467503309249878
epoch:386
Validation Loss:0.14543196558952332
epoch:389
Validation Loss:0.14453041553497314
epoch:392
Validation Loss:0.14401619136333466
epoch:395
Validation Loss:0.14370864629745483
epoch:398
Validation Loss:0.14366251230239868
0.073603146
epoch:409
Validation Loss:0.1435108333826065
epoch:412
Validation Loss:0.14276280999183655
epoch:415
Validation Loss:0.142203688621521
epoch:418
Validation Loss:0.1419370025396347
epoch:421
Validation Loss:0.14181400835514069
epoch:429
Validation Loss:0.14104025065898895
epoch:432
Validation Loss:0.1400272250175476
epoch:435
Validation Loss:0.13934436440467834
epoch:438
Validation Loss:0.13897185027599335
epoch:449
Validation Loss:0.13854198157787323
0.07166256
epoch:452
Validation Loss:0.13738369941711426
epoch:455
Validation Loss:0.13650760054588318
epoch:458
Validation Loss:0.13612858951091766
epoch:461
Validation Loss:0.1360994428396225
epoch:495
Validati

epoch:130
Validation Loss:1.3487889766693115
epoch:140
Validation Loss:1.2374635934829712
epoch:142
Validation Loss:1.1831086874008179
0.17416754
epoch:169
Validation Loss:1.1765496730804443
epoch:184
Validation Loss:1.0959397554397583
epoch:190
Validation Loss:1.0568197965621948
epoch:196
Validation Loss:1.0374162197113037
epoch:198
Validation Loss:0.972104012966156
0.157966
epoch:221
Validation Loss:0.8869550824165344
epoch:230
Validation Loss:0.8782939314842224
0.15081011
epoch:252
Validation Loss:0.8528857231140137
epoch:263
Validation Loss:0.8178390264511108
epoch:264
Validation Loss:0.7830316424369812
epoch:267
Validation Loss:0.7559642791748047
epoch:274
Validation Loss:0.7108650207519531
epoch:284
Validation Loss:0.677547812461853
epoch:294
Validation Loss:0.6395736932754517
epoch:297
Validation Loss:0.6255618333816528
epoch:298
Validation Loss:0.60739666223526
0.15081626
epoch:312
Validation Loss:0.5778009295463562
epoch:323
Validation Loss:0.52793288230896
epoch:331
Validatio

...

1.1015298
epoch:154
Validation Loss:2.3158881664276123
1.1007996


C:\Users\1508-T3630\AppData\Local\Temp\ipykernel_38044\3528218776.py:15: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((pred - true) / true))


1.0794263
1.0706314
epoch:343
Validation Loss:2.295619010925293
epoch:346
Validation Loss:2.279989719390869
1.0731924
epoch:368
Validation Loss:2.2759926319122314
epoch:399
Validation Loss:2.2757885456085205
1.0597341
1.0569249
1.0579048
1.0505006
1.0482718
1.0446526
1.0422583
epoch:738
Validation Loss:2.245832681655884
1.0410575
1.0379189
1.0334654
1.0337112
1.0338442
1.031803
the order1.1finished!
1.942626
1.9234401
1.9143615
1.9050245
1.9024835
1.8985268
epoch:330
Validation Loss:9.668495178222656
1.9000666
1.8939778
epoch:448
Validation Loss:9.576931953430176
1.8986183
1.8992829
1.8999032
1.9015371
1.9083042
1.9068558
1.9099036
1.9121965
1.9128778
1.9221259
1.9215765


C:\Users\1508-T3630\AppData\Local\Temp\ipykernel_38044\3528218776.py:15: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((pred - true) / true))


1.9293194
the order0.7finished!
1.9239727
1.9047687
1.8879187
epoch:172
Validation Loss:9.645217895507812
1.8795849
epoch:244
Validation Loss:9.620336532592773
1.8758609
epoch:298
Validation Loss:9.596565246582031
1.8727946
epoch:305
Validation Loss:9.346423149108887
epoch:318
Validation Loss:9.322710037231445
1.8666527
epoch:353
Validation Loss:9.246650695800781
epoch:361
Validation Loss:8.533973693847656
1.8667696
1.863604
1.8610057
1.8674366
1.864961
1.8680241
1.8632228
epoch:736
Validation Loss:8.462231636047363
1.8628058
1.8624263
1.8660862
1.8629947
1.8642073
1.867335
the order0.8finished!
1.9123892
1.892287
1.8825184
1.8703921
epoch:241
Validation Loss:9.810179710388184
1.8631859
epoch:283
Validation Loss:9.721500396728516
epoch:297
Validation Loss:9.610461235046387
1.8568271
epoch:325
Validation Loss:9.596476554870605
epoch:345
Validation Loss:9.320801734924316
1.8514583
1.8573128
epoch:402
Validation Loss:9.280823707580566
1.8446709
epoch:452
Validation Loss:8.86937141418457
1